# NPI-6: NPI is released (1- or 2-)

Description: check if there is any NPI out of this range, currently only 1- and 2- NPI released. 9999999x is another valid category.

Starting Author: Amy Jin (amy@careset.com)

Date: April 30th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [2]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) col_name:               column to test
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, test will print out all distinct failed NPIs.


def npi_6(db_name, table_name, col_name):
    
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        query = ('''
            SELECT *
            FROM {db}.{t1}
            WHERE ({col1} NOT LIKE '1%') 
            AND ({col1} NOT LIKE '2%') 
            AND ({col1} NOT LIKE '9999999%')
            LIMIT 1;
        '''.format(db = db_name, t1 = table_name, col1 = col_name))

        cur.execute(query)
        rows = list(sum(cur.fetchall(), ()))

        
        if not len(rows):
            print ("Test result: PASS")
        else:
            print ("Test result: FAIL" + '\n')
            print ("The following {} fail the test:".format(col_name) + '\n')
            

            # MySQL query to get failed NPIs
            query = ('''
                SELECT DISTINCT {col1}
                FROM {db}.{t1}
                WHERE ({col1} NOT LIKE '1%') 
                AND ({col1} NOT LIKE '2%') 
                AND ({col1} NOT LIKE '9999999%');
            '''.format(db = db_name, t1 = table_name, col1 = col_name))

            cur.execute(query)
            
            for row in cur.fetchall():
                for i in range(0,len(row)):
                    print (str(row[i]), end=", ")
                print ('\n')
            
        cur.close()
        connection.close()

## Test Example

In [3]:
npi_6('client_celgene',' HCP_Geographic_distribution_of_MM_Patients', '`Physician NPI Number`')

Test result: PASS


In [4]:
npi_6('_amy', 'test_data_good', 'npi_dest')

Test result: PASS


In [5]:
npi_6('_amy', 'test_data_bad1', 'npi')

Test result: FAIL

The following npi fail the test:

7177462947, 

-3890512889, 

9166436362, 

3819969782, 

317195182, 

hello, 

6843300165, 

744882010, 

-1034967478, 

6268907505, 

3660126457, 

5340464469, 

4116131320, 

-2977027415, 

-7808887280, 

, 

-1377764121, 

3454721386, 

4861327419, 

6138432272, 

-6740599147, 

4033406332, 

6988961538, 

9861753251, 

6485314174, 

-1680547904, 

-1940208221, 

8773040198, 

6063590128, 

-3733149805, 

8012021469, 

5966666714, 

-4788451357, 

-7240162248, 

4744361110, 

5062635265, 

-4946354333, 

-2965820727, 

-6306254008, 

-422312163, 

-3674108797, 

8530606963, 

-1562555165, 

-5310822741, 

8317253292, 

-639483141, 

8017443721, 

-1941542794, 

-6736649911, 

-4429541401, 

507389324, 

6171673407, 

4024535856, 

5450168345, 

-7276831325, 

9394164785, 

9057452247, 

-6374967444, 

5720075225, 

9850389340, 

3853438551, 

5732036571, 

-2794293886, 

3758123756, 

-5778586681, 

5441255319, 

7864771591, 

596